# Importation des packages

In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import re
import string

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier  # Ajout pour multi-label
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional



from transformers import DistilBertTokenizer, TFDistilBertModel
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Layer
from tensorflow.keras.optimizers import Adam
import joblib





In [ ]:
pip install wordcloud

# Importation des données

Ajoutez un raccourci de ce dossier à votre google drive :

https://drive.google.com/drive/folders/1mx-CAzT10YKrmxHfYDP_1Oef7PVGUr7s?usp=sharing

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/data_classification_commentaires_toxiques/train.csv')
data.head()

# Etude du jeu de données
Avant de commencer à créer les modèles, il est important de comprendre les données que nous avons afin de mieux prévoir la suite et donc la création des divers modèles

---
***Pour commencer, on récupère les infos du jeu de données***


In [ ]:
print(data.info())

***On s'occupe maintenant de la répartition des différentes catégories de toxicité dans notre jeu de données.***

Cela permet de savoir si les classes sont équilibrées ou déséquilibrées et d’adapter ensuite nos stratégies de modélisation en conséquence.

In [ ]:
#reparition des classes par catégorie de toxicité

# Compter les occurrences de chaque classe
label_counts = data.iloc[:, 2:].sum()

# Affichage en barres
plt.figure(figsize=(8,5))
label_counts.plot(kind='bar', color=['blue', 'red', 'green', 'purple', 'orange', 'brown'])
plt.title("Nombre de commentaires par catégorie de toxicité")
plt.show()


notre jeu de données est constitué de commentaires jugés non toxiques :


In [ ]:
#voir le pourcentage de commentaires jugés non toxiques

clean_comments = (data.iloc[:, 2:].sum(axis=1) == 0).mean()
print(f"Pourcentage de commentaires non toxiques: {clean_comments:.2%}")


On affiche des exemples de commentaires pour pouvoir avoir en plus une idée visuel des types d'insultes, menaces, etc...

In [ ]:
#voir des exemples de commentaires (1 toxique et 1 non toxique)

# Exemple d'un commentaire toxique
print(data[data["toxic"] == 1]["comment_text"].sample(5).values)

# Exemple d'un commentaire propre
print(data[(data.iloc[:, 2:].sum(axis=1) == 0)]["comment_text"].sample(5).values)


In [ ]:
#vérifier qu'aucune donnée n'est manquante

print(data.isnull().sum())


In [ ]:
#analyse des longueurs de commentaires

data["comment_length"] = data["comment_text"].apply(len)

plt.figure(figsize=(8,5))
data["comment_length"].hist(bins=50, color='blue', alpha=0.7)
plt.title("Distribution des longueurs des commentaires")
plt.xlabel("Longueur du commentaire")
plt.ylabel("Nombre de commentaires")
plt.show()


In [ ]:
#faire un nuage de mot pour voir les mots toxiques récurrents

from wordcloud import WordCloud

toxic_comments = " ".join(data[data["toxic"] == 1]["comment_text"])
wordcloud = WordCloud(width=800, height=400, background_color="black").generate(toxic_comments)

plt.figure(figsize=(10,5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title("Nuage de mots des commentaires toxiques")
plt.show()


# Préparation des données

Nous avons pu analyser les données et donc voir ce que nous avons dans notre jeu de données.

Nous avons maintenant une colonne contenant des commentaires sous forme de texte brut, et pour pouvoir les utiliser efficacement dans nos analyses, nous devons les nettoyer.

---

**Pourquoi nettoyer le texte ?**

Le nettoyage du texte permet de simplifier les données en éliminant des éléments qui ne sont pas pertinents pour l'analyse. Par exemple, des éléments comme les liens URL, les mentions de profils sur les réseaux sociaux (comme @username), les hashtags (#motcle), ou encore les ponctuations, ne nous aident généralement pas à comprendre le sens du texte. L'objectif est de nous concentrer sur les mots porteurs de sens.

---

**Comment procédons-nous pour nettoyer le texte ?**

On met en minuscules, nous commençons par transformer tout le texte en minuscules. Cela garantit que nous traitons "Bonjour" et "bonjour" de la même manière.

On s'occupe aussi de supprimer les liens, les mentions et les hashtags, ils n'ont pas d'utilités pour ce projet et nous cherchons à nous concentrer sur les commentaires toxiques.

On supprime la ponctuation et les chiffres.

---

**Tokenisation :**

Ensuite, nous divisons le texte en mots, un processus appelé "tokenisation". Cela nous permet de traiter chaque mot individuellement, facilitant ainsi les analyses ultérieures.

Suppression des "stopwords" plus précisement certains mots très fréquents, comme "et", "le", "la", "is", "in", etc., sont retirés car ils n'apportent pas de valeur significative à l'analyse.

In [ ]:
#nettoyage du texte


nltk.download('punkt')
nltk.download('punkt_tab') # Download the punkt_tab data
nltk.download('stopwords')  # Add this line to download stopwords



STOPWORDS = set(stopwords.words("english"))

def clean_text(text):
    # Mettre en minuscules
    text = text.lower()
    # Supprimer les liens
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    # Supprimer les mentions et hashtags
    text = re.sub(r'\@\w+|\#','', text)
    # Supprimer la ponctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Supprimer les chiffres
    text = re.sub(r'\d+', '', text)
    # Tokenization
    words = word_tokenize(text)
    # Supprimer les stopwords
    words = [word for word in words if word not in STOPWORDS]
    return " ".join(words)

# Appliquer le nettoyage
data["clean_text"] = data["comment_text"].apply(clean_text)

# Vérifier le résultat
print(data[["comment_text", "clean_text"]].head())

**Stemming :**

Cela consiste à réduire un mot à sa racine, en coupant les suffixes. Par exemple, "running" devient "run". Cette méthode est souvent plus rapide mais peut parfois donner des résultats moins précis.


\
**Lemmatisation :**

La lemmatisation est un processus plus sophistiqué qui transforme un mot en son "lemme" (forme canonique) en prenant en compte son sens et sa position grammaticale. Par exemple, "better" sera transformé en "good", et "running" sera réduit à "run" en tant que verbe.

In [ ]:
#stemming


nltk.download("wordnet")

stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    words = word_tokenize(text)
    words = [lemmatizer.lemmatize(word) for word in words]  # Lemmatisation
    return " ".join(words)

# Appliquer la transformation
data["processed_text"] = data["clean_text"].apply(preprocess_text)


On représenter les mots sous forme de vecteurs numériques, ce qui permet de capturer leurs relations sémantiques. Word2Vec est un modèle d'apprentissage qui génère ces représentations vectorielles.

In [ ]:
#word embeddings pour representer le texte
!pip install gensim

from gensim.models import Word2Vec

tokenized_sentences = [word_tokenize(text) for text in data["processed_text"]]
word2vec_model = Word2Vec(sentences=tokenized_sentences, vector_size=100, window=5, min_count=2, workers=4)

# Exemple de représentation pour "toxic"
print(word2vec_model.wv["toxic"])

Comme les commentaires "toxiques" sont beaucoup plus fréquentes que d'autres. Cela peut biaiser les modèles d'apprentissage automatique. Pour corriger cela, nous utilisons les poids de classe.

In [ ]:
#gestion des desequilibre des classes


labels = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

# Get unique classes from the target variable
unique_classes = np.unique(data[labels].values) # Get unique classes from all target columns

# Calculer les poids des classes
class_weights = compute_class_weight(class_weight="balanced", classes=unique_classes, y=data[labels].values.ravel())
print(class_weights)

# Entraînement du modèle baseline

Nous allons utiliser 2 techniques, la première sera du random forest et pour la deuxième nous utiliserons BERT de Google

####Random Forest


In [ ]:
# Random Forest

# Random Forest avec Pipeline


# Charger les données (assumons que 'data' est déjà défini)
# Prendre un échantillon aléatoire de 10% des données
sample_df = data.sample(frac=0.5, random_state=42)

# Features et labels
X = sample_df["processed_text"]
y = sample_df[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]]

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Création de la pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000)),  # Étape 1 : Vectorisation TF-IDF
    ('rf', MultiOutputClassifier(RandomForestClassifier(n_estimators=200, random_state=42)))  # Étape 2 : Modèle Random Forest multi-label
])

# Entraînement de la pipeline
pipeline.fit(X_train, y_train)

# Prédictions
y_pred = pipeline.predict(X_test)

# Évaluation
print(classification_report(y_test, y_pred, target_names=["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]))

# Sauvegarde de la pipeline
import joblib
joblib.dump(pipeline, "rf_toxicity_pipeline.pkl")
print("Pipeline sauvegardée !")


In [ ]:

# Prédictions
y_pred = pipeline.predict(X_test)

# Étiquettes de sortie
labels = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

# Initialisation de la figure
plt.figure(figsize=(15, 12))

# Itérer sur chaque label pour générer la matrice de confusion
for i, label in enumerate(labels):
    plt.subplot(2, 3, i+1)

    # Calcul de la matrice de confusion pour chaque étiquette
    cm = confusion_matrix(y_test[label], y_pred[:, i])

    # Visualisation avec seaborn
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["0", "1"], yticklabels=["0", "1"])

    # Ajouter le titre et les labels
    plt.title(f'Matrice de Confusion - {label}')
    plt.xlabel('Prédictions')
    plt.ylabel('Vraies étiquettes')

plt.tight_layout()
plt.show()


In [ ]:
def predict_toxicity(text):
    # Charger la pipeline
    pipeline = joblib.load("rf_toxicity_pipeline.pkl")

    # Prédire directement sur le texte brut
    prediction = pipeline.predict([text])[0]  # [text] car predict attend une liste

    # Convertir en dictionnaire
    labels = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
    results = {label: int(pred) for label, pred in zip(labels, prediction)}

    return results

# Test interactif
while True:
    user_text = input("Entrez un message (ou 'exit' pour quitter) : ")
    if user_text.lower() == "exit":
        break
    prediction = predict_toxicity(user_text)
    print("\nPrédiction :", prediction)

# Itération de la modélisation

# Version secondaire de BERT (DistilBERT) ancien a ne pas lancer



In [20]:
# Charger les données
sample_df = data.sample(frac=0.5, random_state=42)
X = sample_df["processed_text"].values
y = sample_df[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values

# Paramètres
MAX_VOCAB_SIZE = 20000
MAX_SEQUENCE_LENGTH = 100  # Taille max des séquences
EMBEDDING_DIM = 100  # Taille des embeddings

# Tokenization et transformation en séquences
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, oov_token="<OOV>")
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)
X_padded = pad_sequences(X_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post")

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)

# Construction du modèle LSTM
model = Sequential([
    Embedding(MAX_VOCAB_SIZE, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH),
    Bidirectional(LSTM(128, return_sequences=True)),  # LSTM bidirectionnel
    Dropout(0.3),
    Bidirectional(LSTM(64)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(6, activation='sigmoid')  # 6 sorties pour la classification multi-label
])

# Compilation du modèle
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entraînement du modèle
history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

# Sauvegarde du modèle et du tokenizer
model.save("lstm_toxicity_model.keras")  # Format natif Keras (recommandé)

import pickle
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

print("Modèle et tokenizer sauvegardés !")

# Fonction de prédiction
def predict_toxicity(text):
    model = tf.keras.models.load_model("lstm_toxicity_model.keras")


    # Charger le tokenizer
    with open("tokenizer.pkl", "rb") as f:
        tokenizer = pickle.load(f)

    # Tokenization et padding
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post")

    # Prédiction
    prediction = model.predict(padded_sequence)[0]
    labels = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
    results = {label: float(pred) for label, pred in zip(labels, prediction)}

    return results

# Test interactif
while True:
    user_text = input("Entrez un message (ou 'exit' pour quitter) : ")
    if user_text.lower() == "exit":
        break
    prediction = predict_toxicity(user_text)
    print("\nPrédiction :", prediction)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
1995/1995 ━━━━━━━━━━━━━━━━━━━━ 1459s 726ms/step - accuracy: 0.9197 - loss: 0.1149 - val_accuracy: 0.9932 - val_loss: 0.0565
Epoch 2/5
1995/1995 ━━━━━━━━━━━━━━━━━━━━ 1353s 678ms/step - accuracy: 0.9945 - loss: 0.0487 - val_accuracy: 0.9932 - val_loss: 0.0532
Epoch 3/5
1995/1995 ━━━━━━━━━━━━━━━━━━━━ 1432s 718ms/step - accuracy: 0.9938 - loss: 0.0432 - val_accuracy: 0.9932 - val_loss: 0.0544
Epoch 4/5
1995/1995 ━━━━━━━━━━━━━━━━━━━━ 1447s 711ms/step - accuracy: 0.9924 - loss: 0.0390 - val_accuracy: 0.9932 - val_loss: 0.0569
Epoch 5/5
1995/1995 ━━━━━━━━━━━━━━━━━━━━ 1498s 728ms/step - accuracy: 0.9902 - loss: 0.0341 - val_accuracy: 0.9931 - val_loss: 0.0624
Modèle et tokenizer sauvegardés !


In [24]:
# Fonction de prédiction
def predict_toxicity(text):
    model = tf.keras.models.load_model("lstm_toxicity_model.keras")


    # Charger le tokenizer
    with open("tokenizer.pkl", "rb") as f:
        tokenizer = pickle.load(f)

    # Tokenization et padding
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post")

    # Prédiction
    prediction = model.predict(padded_sequence)[0]
    labels = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
    results = {label: float(pred) for label, pred in zip(labels, prediction)}

    return results

# Test interactif
while True:
    user_text = input("Entrez un message (ou 'exit' pour quitter) : ")
    if user_text.lower() == "exit":
        break
    prediction = predict_toxicity(user_text)
    print("\nPrédiction :", prediction)


Entrez un message (ou 'exit' pour quitter) : exit


# modele 2



# Nouveau mais entrainement long


In [26]:
sample_df = data.sample(frac=0.5, random_state=42)
X = sample_df["processed_text"].values
y = sample_df[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values

# Initialiser le tokenizer BERT
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

def encode_texts(texts, tokenizer, max_len=128):
    return tokenizer(texts.tolist(), truncation=True, padding='max_length', max_length=max_len, return_tensors='tf')

# Tokenisation des textes
encoded_texts = encode_texts(X, tokenizer)
input_ids = encoded_texts['input_ids']
attention_masks = encoded_texts['attention_mask']

# Split train/test
X_train, X_test, mask_train, mask_test, y_train, y_test = train_test_split(
    input_ids.numpy(), attention_masks.numpy(), y, test_size=0.2, random_state=42
)

# Définir une couche personnalisée pour encapsuler TFDistilBertModel
class TFDistilBertLayer(Layer):
    def __init__(self, bert_model, **kwargs):
        super(TFDistilBertLayer, self).__init__(**kwargs)
        self.bert_model = bert_model

    def call(self, inputs, **kwargs):
        input_ids, attention_mask = inputs
        outputs = self.bert_model(input_ids=input_ids, attention_mask=attention_mask)
        return outputs[0][:, 0, :]  # Retourne le CLS token

    def get_config(self):
        config = super(TFDistilBertLayer, self).get_config()
        # Pas besoin de sérialiser bert_model directement, il sera chargé séparément
        return config

# Charger le modèle BERT pré-entraîné
bert_model = TFDistilBertModel.from_pretrained("distilbert-base-uncased")
bert_model.trainable = False  # Geler les poids de BERT (optionnel)

# Définir les entrées avec l'API fonctionnelle de Keras
input_ids_layer = Input(shape=(128,), dtype=tf.int32, name="input_ids")
attention_mask_layer = Input(shape=(128,), dtype=tf.int32, name="attention_mask")

# Utiliser la couche personnalisée
bert_output = TFDistilBertLayer(bert_model)([input_ids_layer, attention_mask_layer])

# Ajouter des couches supplémentaires
dense = Dense(128, activation='relu')(bert_output)
dropout = Dropout(0.3)(dense)
output = Dense(6, activation='sigmoid')(dropout)  # 6 classes pour multi-label

# Créer le modèle
model = Model(inputs=[input_ids_layer, attention_mask_layer], outputs=output)

# Compilation du modèle
optimizer = Adam(learning_rate=2e-5)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Entraînement
epochs = 4
batch_size = 16
model.fit(
    [X_train, mask_train], y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=([X_test, mask_test], y_test)
)

# Prédictions avec un seuil ajusté
y_pred = (model.predict([X_test, mask_test]) > 0.3).astype(int)  # Seuil abaissé à 0.3
print(classification_report(y_test, y_pred, target_names=["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"], zero_division=0))

# Sauvegarde du modèle au format natif Keras
model.save("bert_toxicity_model.keras")  # Format recommandé
joblib.dump(tokenizer, "bert_tokenizer.pkl")
print("Modèle et tokenizer sauvegardés !")

# Fonction de prédiction
def predict_toxicity(text):
    tokenizer = joblib.load("bert_tokenizer.pkl")
    model = tf.keras.models.load_model("bert_toxicity_model.keras", custom_objects={'TFDistilBertLayer': TFDistilBertLayer})

    encoded = encode_texts(np.array([text]), tokenizer)
    input_ids, attention_mask = encoded['input_ids'], encoded['attention_mask']
    prediction = (model.predict([input_ids, attention_mask]) > 0.3).astype(int)[0]  # Seuil ajusté

    labels = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
    return {label: int(pred) for label, pred in zip(labels, prediction)}

# Test interactif
while True:
    user_text = input("Entrez un message (ou 'exit' pour quitter) : ")
    if user_text.lower() == "exit":
        break
    print("\nPrédiction :", predict_toxicity(user_text))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Epoch 1/4
  15/3990 ━━━━━━━━━━━━━━━━━━━━ 4:25:04 4s/step - accuracy: 0.6286 - loss: 0.6318

KeyboardInterrupt: 